The dynamics are given by

$$\begin{align}\ddot{\mathbf{r}} &= \begin{bmatrix}0\\0\\-g\end{bmatrix} + \mathbf{R} \begin{bmatrix}0\\0\\\frac{F_{tot}}{m}\end{bmatrix}\\ \dot{\omega}_{BW} &= \mathbf{I}^{-1} \left(-\omega_{BW} \times \mathbf{I} \omega_{BW} + \begin{bmatrix}M_1\\ M_2\\ M_3\end{bmatrix}\right)\end{align}$$

where $\mathbf{r}$ is the position in the world frame of the center of mass of the quadrotor, i.e.

$$\mathbf{r} = \begin{bmatrix}x\\ y\\ z\end{bmatrix}$$

the rotation matrix $\mathbf{R}$ maps from the body frame to the world frame following the Space 1-2-3 convention, that is

$$\mathbf{R} = \begin{bmatrix}\cos(\theta)\cos(\psi) & \sin(\phi)\sin(\theta)\cos(\psi) - \sin(\psi)\cos(\phi) & \cos(\phi)\sin(\theta)\cos(\psi) + \sin(\psi)\sin(\phi)\\
\cos(\theta)\sin(\psi) & \sin(\phi)\sin(\theta)\cos(\psi) + \cos(\psi)\cos(\phi) & \cos(\phi)\sin(\phi)\sin(\psi) - \cos(\theta)\sin(\phi)\\
-\sin(\theta) & \sin(\phi)\cos(\theta) & \cos(\phi)\cos(\theta)\end{bmatrix}$$

and 

$$\omega_{BW} = \begin{bmatrix}p\\ q\\ r\end{bmatrix}$$

is the angular velocity of the body frame in the world frame, which is related to the rate of change of the Euler angles by

$$\begin{bmatrix}\dot{\phi}\\ \dot{\theta}\\ \dot{\psi}\end{bmatrix} = \begin{bmatrix}1 & \sin(\phi)\tan(\theta) & \cos(\phi)\tan(\theta)\\
         0 & \cos(\phi) & -\sin(\phi)\\
         0 & \frac{\sin(\phi)}{\cos(\theta)} & \frac{\cos(\phi)}{\cos(\theta)}
        \end{bmatrix}\begin{bmatrix}p\\ q\\ r\end{bmatrix}$$

In addition to the hover state and input, the following values for the physical parameters of the system are substituted:

- $m = 0.03kg$
- $g = 9.81 \frac{m}{s^2}$
- $I_{xx} = 1.4194e-05$
- $I_{yy} = 1.4089e-05$
- $I_{zz} = 2.9741e-05$

Note the full state of the system $\mathbf{x}$ and system inputs $\mathbf{u}$ are the variables, in order,

$$\begin{align}\mathbf{x} = \begin{bmatrix}x\\ y\\ z\\ \phi\\ \theta\\ \psi\\ \dot{x}\\ \dot{y}\\ \dot{z}\\ p\\ q\\ r\end{bmatrix} & & \mathbf{u} = \begin{bmatrix}F_{tot}\\ M_1\\ M_2\\ M_3\end{bmatrix}\end{align}$$

and the hover state for this system that we are linearizing about is:

$$\begin{align}\mathbf{x} = \mathbf{0} & & \mathbf{u} = \begin{bmatrix}mg\\ 0\\ 0\\ 0\end{bmatrix}\end{align}$$

In [1]:
import sympy as sp
import numpy as np
from sympy.physics.vector import dynamicsymbols as dynamicsymbols
from sympy import cos,sin,tan
from scipy.linalg import solve_continuous_are

In [2]:
def initialize_quadrotor(Q, R):
    
    m,g,Ixx,Iyy,Izz,t=sp.symbols('m g Ixx Iyy Izz t')
    u1,u2,u3,u4=sp.symbols('u1 u2 u3 u4')
    x,y,z,phi,theta,psi=dynamicsymbols('x y z phi theta psi')
    
    x_dot=sp.diff(x,t)
    y_dot=sp.diff(y,t)
    z_dot=sp.diff(z,t)
    phi_dot=sp.diff(phi,t)
    theta_dot=sp.diff(theta,t)
    psi_dot=sp.diff(psi,t)
    
    p,q,r=dynamicsymbols('p q r')
    omega=sp.Matrix([p,q,r])
    conv=sp.Matrix([[1,sin(phi)*tan(theta),cos(phi)*tan(theta)],
                    [0,cos(phi),-sin(phi)],
                    [0,sin(phi)/cos(theta),cos(phi)/cos(theta)]])
    euler_dot=conv*omega
    
    rotation_matrix=sp.Matrix([[cos(theta)*cos(psi),sin(phi)*sin(theta)*cos(psi)-sin(psi)*cos(phi),cos(phi)*sin(theta)*cos(psi)+sin(psi)*sin(phi)],
                               [cos(theta)*sin(psi),sin(phi)*sin(theta)*cos(psi)+cos(psi)*cos(phi),cos(phi)*sin(phi)*sin(psi)-cos(theta)*sin(phi)],
                               [-sin(theta),sin(phi)*cos(theta),cos(phi)*cos(theta)]])
    acceleration=sp.Matrix([0,0,-g])+rotation_matrix*sp.Matrix([0,0,u1/m])
    
    I=sp.Matrix([[Ixx,0,0],[0,Iyy,0],[0,0,Izz]])
    omega_dot=(I.inv())*(-omega.cross(I*omega)+sp.Matrix([u2,u3,u4]))
    
    state=sp.Matrix([x,y,z,phi,theta,psi,x_dot,y_dot,z_dot,p,q,r])
    input=sp.Matrix([u1,u2,u3,u4])
    dynamics=sp.Matrix([x_dot,y_dot,z_dot,euler_dot,acceleration,omega_dot])
    
    A=dynamics.jacobian(state)
    B=dynamics.jacobian(input)
    
    K = lqr_controller(A, B, Q, R)

    return A, B, K

In [3]:
def lqr_controller(A, B, Q, R):
    fixed_point = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    fixed_point_input = [m * g, 0, 0, 0]

    # Convert symbolic expressions to numerical values
    A_arr = np.array(A.subs([(x, fixed_point[0]), (y, fixed_point[1]), (z, fixed_point[2]),
                             (phi, fixed_point[3]), (theta, fixed_point[4]), (psi, fixed_point[5]),
                             (x_dot, fixed_point[6]), (y_dot, fixed_point[7]), (z_dot, fixed_point[8]),
                             (p, fixed_point[9]), (q, fixed_point[10]), (r, fixed_point[11]),
                             (u1, fixed_point_input[0]), (u2, fixed_point_input[1]), (u3, fixed_point_input[2]),
                             (u4, fixed_point_input[3]), (m, 0.03), (g, 9.81),
                             (Ixx, 1.4194 * (10 ** -5)), (Iyy, 1.4089 * (10 ** -5)), (Izz, 2.9741 * (10 ** -5))]))

    B_arr = np.array(B.subs([(x, fixed_point[0]), (y, fixed_point[1]), (z, fixed_point[2]),
                             (phi, fixed_point[3]), (theta, fixed_point[4]), (psi, fixed_point[5]),
                             (x_dot, fixed_point[6]), (y_dot, fixed_point[7]), (z_dot, fixed_point[8]),
                             (p, fixed_point[9]), (q, fixed_point[10]), (r, fixed_point[11]),
                             (u1, fixed_point_input[0]), (u2, fixed_point_input[1]), (u3, fixed_point_input[2]),
                             (u4, fixed_point_input[3]), (m, 0.03), (g, 9.81),
                             (Ixx, 1.4194 * (10 ** -5)), (Iyy, 1.4089 * (10 ** -5)), (Izz, 2.9741 * (10 ** -5))]))

    # Compute control gains using LQR
    K = solve_continuous_are(A_arr, B_arr, Q, R)

    return K

In [4]:
def update_dynamics(A,B,current_state,control_imput):
    
        A_arr=A.subs([(x,present_state[0]),(y,present_state[1]),(z,present_state[2]),
            (phi,present_state[3]),(theta,present_state[4]),(psi,present_state[5]),
            (x_dot,present_state[6]),(y_dot,present_state[7]),(z_dot,present_state[8]),
            (p,present_state[9]),(q,present_state[10]),(r,present_state[11]),
            (u1,control_input[0]),(u2,control_input[1]),(u3,control_input[2]),(u4,control_input[3]),
            (m,0.03),(g,9.81),(Ixx,1.4194*(10**-5)),(Iyy,1.4089*(10**-5)),(Izz,2.9741*(10**-5))])
        
        B_arr=B.subs([(x,present_state[0]),(y,present_state[1]),(z,present_state[2]),
            (phi,present_state[3]),(theta,present_state[4]),(psi,present_state[5]),
            (x_dot,present_state[6]),(y_dot,present_state[7]),(z_dot,present_state[8]),
            (p,present_state[9]),(q,present_state[10]),(r,present_state[11]),
            (u1,control_input[0]),(u2,control_input[1]),(u3,control_input[2]),(u4,control_input[3]),
            (m,0.03),(g,9.81),(Ixx,1.4194*(10**-5)),(Iyy,1.4089*(10**-5)),(Izz,2.9741*(10**-5))])
        
        return A_arr,B_arr

In [5]:
def simulate_quadrotor(A, B, K, duration, dt):
    
    # Initial state
    initial_state = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0]) 

    # Initialize lists to store the trajectory and control inputs
    trajectory = [initial_state]
    control_inputs = []

    # Simulation loop
    t = 0.0
    while t < duration:
        # Get the current state
        current_state = trajectory[-1]

        # Compute the control input using the LQR controller
        control_input = -np.dot(K, current_state)

        # Store the control input
        control_inputs.append(control_input)

        # Update the state using quadrotor dynamics
        new_state = update_dynamics(A,B,current_state, control_input)

        # Append the new state to the trajectory
        trajectory.append(new_state)

        # Update the time
        t += dt

    return np.array(trajectory), np.array(control_inputs)

In [6]:
m,g,Ixx,Iyy,Izz,t=sp.symbols('m g Ixx Iyy Izz t')
u1,u2,u3,u4=sp.symbols('u1 u2 u3 u4')
x,y,z,phi,theta,psi=dynamicsymbols('x y z phi theta psi')

In [7]:
x_dot=sp.diff(x,t)
y_dot=sp.diff(y,t)
z_dot=sp.diff(z,t)
phi_dot=sp.diff(phi,t)
theta_dot=sp.diff(theta,t)
psi_dot=sp.diff(psi,t)

In [8]:
p,q,r=dynamicsymbols('p q r')
omega=sp.Matrix([p,q,r])
conv=sp.Matrix([[1,sin(phi)*tan(theta),cos(phi)*tan(theta)],
                [0,cos(phi),-sin(phi)],
                [0,sin(phi)/cos(theta),cos(phi)/cos(theta)]])
euler_dot=conv*omega
print(euler_dot)

Matrix([[p(t) + q(t)*sin(phi(t))*tan(theta(t)) + r(t)*cos(phi(t))*tan(theta(t))], [q(t)*cos(phi(t)) - r(t)*sin(phi(t))], [q(t)*sin(phi(t))/cos(theta(t)) + r(t)*cos(phi(t))/cos(theta(t))]])


In [9]:
rotation_matrix=sp.Matrix([[cos(theta)*cos(psi),sin(phi)*sin(theta)*cos(psi)-sin(psi)*cos(phi),cos(phi)*sin(theta)*cos(psi)+sin(psi)*sin(phi)],
                           [cos(theta)*sin(psi),sin(phi)*sin(theta)*cos(psi)+cos(psi)*cos(phi),cos(phi)*sin(phi)*sin(psi)-cos(theta)*sin(phi)],
                           [-sin(theta),sin(phi)*cos(theta),cos(phi)*cos(theta)]])
acceleration=sp.Matrix([0,0,-g])+rotation_matrix*sp.Matrix([0,0,u1/m])
print(acceleration)

Matrix([[u1*(sin(phi(t))*sin(psi(t)) + sin(theta(t))*cos(phi(t))*cos(psi(t)))/m], [u1*(sin(phi(t))*sin(psi(t))*cos(phi(t)) - sin(phi(t))*cos(theta(t)))/m], [-g + u1*cos(phi(t))*cos(theta(t))/m]])


In [10]:
I=sp.Matrix([[Ixx,0,0],[0,Iyy,0],[0,0,Izz]])
omega_dot=(I.inv())*(-omega.cross(I*omega)+sp.Matrix([u2,u3,u4]))

In [11]:
state=sp.Matrix([x,y,z,phi,theta,psi,x_dot,y_dot,z_dot,p,q,r])
input=sp.Matrix([u1,u2,u3,u4])
dynamics=sp.Matrix([x_dot,y_dot,z_dot,euler_dot,acceleration,omega_dot])

In [12]:
A=dynamics.jacobian(state)
B=dynamics.jacobian(input)

In [13]:
A.subs([(x,0),(y,0),(z,0),
        (phi,0),(theta,0),(psi,0),
        (x_dot,0),(y_dot,0),(z_dot,0),
        (p,0),(q,0),(r,0),
        (u1,m*g),(u2,0),(u3,0),(u4,0),
        (m,0.03),(g,9.81),(Ixx,1.4194*(10**-5)),(Iyy,1.4089*(10**-5)),(Izz,2.9741*(10**-5))])

Matrix([
[0, 0, 0,     0,    0, 0, 1, 0, 0, 0, 0, 0],
[0, 0, 0,     0,    0, 0, 0, 1, 0, 0, 0, 0],
[0, 0, 0,     0,    0, 0, 0, 0, 1, 0, 0, 0],
[0, 0, 0,     0,    0, 0, 0, 0, 0, 1, 0, 0],
[0, 0, 0,     0,    0, 0, 0, 0, 0, 0, 1, 0],
[0, 0, 0,     0,    0, 0, 0, 0, 0, 0, 0, 1],
[0, 0, 0,     0, 9.81, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, -9.81,    0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0,     0,    0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0,     0,    0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0,     0,    0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0,     0,    0, 0, 0, 0, 0, 0, 0, 0]])

In [14]:
B.subs([(x,0),(y,0),(z,0),
        (phi,0),(theta,0),(psi,0),
        (x_dot,0),(y_dot,0),(z_dot,0),
        (p,0),(q,0),(r,0),
        (u1,m*g),(u2,0),(u3,0),(u4,0),
        (m,0.03),(g,9.81),(Ixx,1.4194*(10**-5)),(Iyy,1.4089*(10**-5)),(Izz,2.9741*(10**-5))])

Matrix([
[               0,                0,                0,                0],
[               0,                0,                0,                0],
[               0,                0,                0,                0],
[               0,                0,                0,                0],
[               0,                0,                0,                0],
[               0,                0,                0,                0],
[               0,                0,                0,                0],
[               0,                0,                0,                0],
[33.3333333333333,                0,                0,                0],
[               0, 70452.3037903339,                0,                0],
[               0,                0, 70977.3582227269,                0],
[               0,                0,                0, 33623.6172287415]])

In [15]:
A_arr=np.array(A.subs([(x,0),(y,0),(z,0),
        (phi,0),(theta,0),(psi,0),
        (x_dot,0),(y_dot,0),(z_dot,0),
        (p,0),(q,0),(r,0),
        (u1,m*g),(u2,0),(u3,0),(u4,0),
        (m,0.03),(g,9.81),(Ixx,1.4194*(10**-5)),(Iyy,1.4089*(10**-5)),(Izz,2.9741*(10**-5))])).astype(float)
B_arr=np.array(B.subs([(x,0),(y,0),(z,0),
        (phi,0),(theta,0),(psi,0),
        (x_dot,0),(y_dot,0),(z_dot,0),
        (p,0),(q,0),(r,0),
        (u1,m*g),(u2,0),(u3,0),(u4,0),
        (m,0.03),(g,9.81),(Ixx,1.4194*(10**-5)),(Iyy,1.4089*(10**-5)),(Izz,2.9741*(10**-5))])).astype(float)

In [16]:
print(A_arr)
print()
print(B_arr)

[[ 0.    0.    0.    0.    0.    0.    1.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    1.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    1.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    1.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    1.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    1.  ]
 [ 0.    0.    0.    0.    9.81  0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.   -9.81  0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  ]]

[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.0

In [17]:
A=A_arr
B=B_arr

In [18]:
Q=np.eye(12)
R=np.eye(4)

In [19]:
Ks=solve_continuous_are(A,B,Q,R)
print(Ks)

[[ 1.45153773e+00  0.00000000e+00  5.34068720e-18  0.00000000e+00
   1.00007650e+00  0.00000000e+00  5.53480890e-01  0.00000000e+00
   6.93763476e-17  0.00000000e+00  1.40890000e-05  0.00000000e+00]
 [ 0.00000000e+00  1.45153783e+00  0.00000000e+00 -1.00007707e+00
   0.00000000e+00  1.03563319e-15  0.00000000e+00  5.53481043e-01
   0.00000000e+00 -1.41940000e-05  0.00000000e+00 -5.21636091e-18]
 [ 5.34068720e-18  0.00000000e+00  1.02956301e+00  0.00000000e+00
  -2.64361295e-17  0.00000000e+00 -1.62647194e-17  0.00000000e+00
   3.00000000e-02  0.00000000e+00 -6.31747351e-19  0.00000000e+00]
 [ 0.00000000e+00 -1.00007707e+00  0.00000000e+00  5.42986535e+00
   0.00000000e+00 -2.38700374e-15  0.00000000e+00 -1.45163550e+00
   0.00000000e+00  7.70684383e-05  0.00000000e+00  5.78460458e-18]
 [ 1.00007650e+00  0.00000000e+00 -2.64361295e-17  0.00000000e+00
   5.42986226e+00  0.00000000e+00  1.45163468e+00  0.00000000e+00
   7.55915132e-17  0.00000000e+00  7.64983041e-05  0.00000000e+00]
 [ 0.

In [20]:
A, B, K = initialize_quadrotor(np.eye(12), np.eye(4))   
trajectory, control_inputs = simulate_quadrotor(A, B, K, 1, 0.1)

ValueError: object arrays are not supported